**PREPROCESSING WITH PRE-FOG**

In [23]:
import pandas as pd
import numpy as np
import os
import copy
import math

In [24]:
def label_prefog(dataset,window_length = 1):
  dataset.drop(index = list(dataset[dataset['Action'] == 0].index),inplace=True)
  window_length = 64*window_length
    
  fog_index=[]
  for i in dataset.index: 
      if dataset.loc[i,'Action'] == 2:
        fog_index.append(i)
  fog_index

  start_indices=[]
  for i in fog_index:
    if (dataset.loc[i-1,'Action']!=dataset.loc[i,'Action']):
      start_indices.append(i)
 

  prefog=[]
  for start in start_indices:
    prefog_start = [x for x in range(start-window_length,start)]
    prefog.append(prefog_start)

  prefog = [item for sublist in prefog for item in sublist]

  for i in prefog:
       dataset.loc[i,'Action'] = 3
  dataset['Action'] = dataset['Action'] - 1
  return dataset

In [25]:
data_path = '/Users/ashwin/Documents/Ashwin/Bright Academy/Research/dataset_fog_release/dataset'

In [26]:
path = os.getcwd()

In [27]:
os.listdir(data_path)

['S10R01.txt',
 'S02R01.txt',
 'S02R02.txt',
 'S06R01.txt',
 'S06R02.txt',
 'S08R01.txt',
 'S04R01.txt',
 'S01R01.txt',
 'S01R02.txt',
 'S03R02.txt',
 'S03R03.txt',
 'S03R01.txt',
 'S07R02.txt',
 'S07R01.txt',
 'S09R01.txt',
 'S05R01.txt',
 'S05R02.txt']

In [28]:
dataset = pd.DataFrame()

In [29]:
people = []
for person in os.listdir(data_path):
    if '.txt' in person:
        people.append(person)
for window_length in range(1,5):
    for person in people:
        name = person.split('R')[0]
        print (name)
        file = data_path+"/"+person
        temp = pd.read_csv(file,delimiter= " ", header = None)
        print (person,' is read',end = '\t')
        if 2 in temp[max(temp.columns)].unique():
            print ('Adding {} to dataset'.format(person),end = '\t')
            temp.columns = ['time','A_F','A_V','A_L','L_F','L_V','L_L','T_F','T_V','T_L','Action']
            temp = label_prefog(temp,window_length).reset_index(drop=True)
            temp['name'] = name
            print ('{} is labelled'.format(person))
            dataset = pd.concat([dataset,temp],axis = 0)

        print ('')
    dataset.reset_index(drop =True,inplace=True)
    to_path = path + "/raw_labelled"
    to_name = to_path +"/win_"+str(window_length)+".csv"

    dataset.to_csv(to_name,index = False)


display(dataset)

S10
S10R01.txt  is read	
S02
S02R01.txt  is read	Adding S02R01.txt to dataset	S02R01.txt is labelled

S02
S02R02.txt  is read	Adding S02R02.txt to dataset	S02R02.txt is labelled

S06
S06R01.txt  is read	Adding S06R01.txt to dataset	S06R01.txt is labelled

S06
S06R02.txt  is read	
S08
S08R01.txt  is read	Adding S08R01.txt to dataset	S08R01.txt is labelled

S04
S04R01.txt  is read	
S01
S01R01.txt  is read	Adding S01R01.txt to dataset	S01R01.txt is labelled

S01
S01R02.txt  is read	Adding S01R02.txt to dataset	S01R02.txt is labelled

S03
S03R02.txt  is read	Adding S03R02.txt to dataset	S03R02.txt is labelled

S03
S03R03.txt  is read	
S03
S03R01.txt  is read	Adding S03R01.txt to dataset	S03R01.txt is labelled

S07
S07R02.txt  is read	Adding S07R02.txt to dataset	S07R02.txt is labelled

S07
S07R01.txt  is read	Adding S07R01.txt to dataset	S07R01.txt is labelled

S09
S09R01.txt  is read	Adding S09R01.txt to dataset	S09R01.txt is labelled

S05
S05R01.txt  is read	Adding S05R01.txt to dataset	

,time,A_F,A_V,A_L,L_F,L_V,L_L,T_F,T_V,T_L,Action,name
0,670000,-151,990,267,63,990,80,-9,1019,0,0,S02
1,670015,-151,1000,277,63,981,80,-9,1009,58,0,S02
2,670031,-181,1009,247,63,981,70,-9,1019,38,0,S02
3,670046,-161,990,237,81,981,80,0,1019,48,0,S02
4,670062,-151,1000,257,81,981,80,-9,1000,38,0,S02
...,...,...,...,...,...,...,...,...,...,...,...,...
3298667,1469937,343,1009,118,818,481,-50,864,485,203,0,S05
3298668,1469953,313,990,128,818,527,-80,844,476,223,0,S05
3298669,1469968,333,1000,108,827,500,-20,864,466,194,0,S05
3298670,1469984,323,1039,108,836,500,-40,854,457,233,0,S05


In [30]:
def create_window(act,window_length,dataframe):
    
  indices = list(dataframe[dataframe.Action == act].index)
  groups = []
  temp = []
  group_count = 0
  for i in range(len(indices)):
    if i == len(indices)-1:
      temp.append(indices[i])
      groups.append(temp)
      temp = []
      break
    temp.append(indices[i])
    if indices[i]+1 != indices[i+1]: 
      group_count+=1
      groups.append(temp)
      temp = []

  fs = 64
  window_length = 1
  # window_length = window_length*fs

  final_dataframe = pd.DataFrame()
  for i in groups: 
    required = math.floor(len(i)/(window_length*fs))
    
    req_index = i[0:(required*fs)]
    
    final_dataframe = pd.concat([final_dataframe,dataframe.iloc[req_index,:]],axis = 0)
  return final_dataframe

In [32]:
for window_length in range(1,5):
  
#   path = os.getcwd()+"/dataset_fog_release/dataset"
  name = path+"/raw_labelled/win_"+str(window_length)+".csv"
  dataframe = pd.read_csv(name)

  activities = []
  for act in range(3):
    activities.append(create_window(act,window_length,dataframe))
  to_write = pd.concat(activities,axis = 0)
  to_path = path + "/windows"+"/windowed_"+str(window_length)+".csv"
  to_write.to_csv(to_path,index = False)

In [33]:
fs = 64
for window_length in range(1,5):
  w = window_length*fs
  FE_path = path + "/windows/windowed_"
  name = FE_path + str(window_length) + ".csv"
  dataframe = pd.read_csv(name)

  df = dataframe.drop(columns=['time','Action','name'])
  stat = pd.DataFrame()


  col= list(df.columns)
  for s in col:
    print (s,end=" ")
    mn =[]
    var = []
    std = []
    mav = []
    rms =[]
    for i in range(0,len(df),w):
        mn_  = np.mean(df[s].iloc[i:i+w])
        var_  = np.var(df[s].iloc[i:i+w])
        std_  = np.std(df[s].iloc[i:i+w])
        mav_  = np.mean(abs(df[s].iloc[i:i+w]))
        rms_  = np.sqrt(np.mean((df[s].iloc[i:i+w])**2))

        mn.append(mn_)
        var.append(var_)
        std.append(std_)
        mav.append(mav_)
        rms.append(rms_)

    stat['mean_'+s] = mn
    stat['var_'+s] = var
    stat['std_'+s] = std
    stat['rms_'+s] = rms
    stat['mav_'+s] = mav


  stat.shape


  import copy
  stat1 = copy.copy(stat)
  stat1['w'] = dataframe['Action'].iloc[[x for x in range(0,len(dataframe),w)]].to_list()
  order = ['w']
  order += stat1.columns.to_list()[:-1]
  stat1 = stat1[order]
  stat1.columns
  col = stat1.columns.to_list()
  col[0] = 0
  stat1.columns = col
  feature_name = path + "/features/time_"+str(window_length)+".csv"
  stat1.to_csv(feature_name, index = False)

A_F A_V A_L L_F L_V L_L T_F T_V T_L A_F A_V A_L L_F L_V L_L T_F T_V T_L A_F A_V A_L L_F L_V L_L T_F T_V T_L A_F A_V A_L L_F L_V L_L T_F T_V T_L 

In [34]:
from scipy.signal import butter, lfilter

#window_length = 3
fs = 64
for window_length in range(1,5):
  w = window_length*fs
  FE_path = path + "/windows/windowed_"
  name = FE_path + str(window_length) + ".csv"
  dataframe = pd.read_csv(name)

  df = dataframe.drop(columns=['time','Action','name'])

  col= list(df.columns)

  order=5

  fi=pd.DataFrame()

  power = pd.DataFrame()
  bands = {'locomotor' :(0.5,3),'freeze' :(3,8)}

  for s in col:
      xtemp = []
      xtemp1 = []
      for i in range(0,len(df),w):
          nyq=0.5*fs

          #locomotor band 0.5-3hz
          loc_low= 0.5/nyq
          loc_high=3/nyq

          #clipping off band from the window
          b, a = butter(order, [loc_low, loc_high], btype='band')
          y=lfilter(b,a,df[s].iloc[i:i+w])

          #total power in locomotor band
          e1=sum([x**2 for x in y])

          #freeze band 3-8hz
          frez_low= 3/nyq
          frez_high=8/nyq

          #clipping off band from the window
          b1, a1 = butter(order, [frez_low, frez_high], btype='band')
          y1=lfilter(b1,a1,df[s].iloc[i:i+w])
          #total power in locomotor band
          e2=sum([x**2 for x in y1])

          FI=e2/e1
          POW=e2+e1
          xtemp.append(FI)
          xtemp1.append(POW)
      fi['FI'+s] = xtemp
      power['P'+s] = xtemp1
  print ("Freeze and power done")


  w = window_length*fs
  E=[]
  for i in range(0,len(df),w):
    energy = np.sum((df.iloc[i:i+w,:])**2)
    E.append(energy)
  E = pd.DataFrame(E)
  E.columns = ["EN_" + x for x in df.columns]

  #Entropy
  from scipy.signal import periodogram

  peak_f = pd.DataFrame()
  PSE = pd.DataFrame()
  for s in col:
    peakF = []
    pse = []
    for i in range(0,len(df),w):
        f,Pxx_den = periodogram(df[s].iloc[i:i+w],fs)
        p_norm = Pxx_den/sum(Pxx_den)
        p_norm = list(filter(lambda a: a != 0, p_norm))
        pse.append(-(np.sum(p_norm*np.log(p_norm))))
        peak = (fs/w)*max(Pxx_den)
        peakF.append(peak)
    PSE['ENt_'+s] = pse
    peak_f['peak_'+s] = peakF
  PSE.fillna(0,inplace = True)


  freq = pd.concat([fi,power,E,PSE,peak_f],axis = 1)

  feature_name = path + "/features/freq_"+str(window_length)+".csv"
  freq.to_csv(feature_name, index = False)

Freeze and power done


/var/folders/wv/v2__c9c92wvg9t0j94btmdzw0000gn/T/ipykernel_79894/3882232058.py:76: RuntimeWarning: invalid value encountered in true_divide
  p_norm = Pxx_den/sum(Pxx_den)


Freeze and power done
Freeze and power done
Freeze and power done


**ML MODELS**

In [35]:
df

,A_F,A_V,A_L,L_F,L_V,L_L,T_F,T_V,T_L
0,-151,990,267,63,990,80,-9,1019,0
1,-151,1000,277,63,981,80,-9,1009,58
2,-181,1009,247,63,981,70,-9,1019,38
3,-161,990,237,81,981,80,0,1019,48
4,-151,1000,257,81,981,80,-9,1000,38
...,...,...,...,...,...,...,...,...,...
3243707,-383,794,277,-63,879,80,58,1038,-87
3243708,-414,725,267,-118,888,101,87,1057,-48
3243709,-252,696,247,-181,879,101,87,1057,-48
3243710,-40,627,168,-209,851,40,77,1057,-48


In [36]:
def normalize(df):
    a = df.drop(columns = ['0'])
    for i in df.columns.to_list():
        mean = df[i].mean()
        std = df[i].std()
        df[i] = (df[i]-mean)/std
    return df

In [37]:
def rfe(model,X_train,y_train):
    from sklearn.feature_selection import RFE
    selector = RFE(model,60,1,1)
    selector.fit(X_train,y_train)
    selector.ranking_
    y_p = selector.predict(X_test)
    print (confusion_matrix(y_test,y_p))
    print (classification_report(y_test,y_p,digits = 5))

In [38]:
def getsize(model):
    p = pickle.dumps(model)
    return sys.getsizeof(p)

In [39]:
def getfeatures(model,X_train,y_train):
    from sklearn.feature_selection import RFE
    model = DecisionTreeClassifier()
    selector = RFE(model,60,1)
    selector.fit(X_train,y_train)
    col = []
    for i in range(len(selector.ranking_)):
        if selector.ranking_[i] == 1:
            col.append(X_train.columns[i])
    return col

In [40]:
featurePath = path + '/features'
windowLength = 1
fs = 64
    
readTime = featurePath +"/time_" +str(window_length)+".csv"
readFreq = featurePath + "/freq_"+str(window_length)+".csv"
timeDom = pd.read_csv(readTime)
freqDom = pd.read_csv(readFreq)

print (timeDom.shape,freqDom.shape)
    
df = pd.concat([timeDom,freqDom],axis = 1)
    
print (df['0'].value_counts())
    
df = df[df['0'] != 2]

print (df['0'].value_counts())

(12671, 46) (12671, 45)
0    10488
1     1594
2      589
Name: 0, dtype: int64
0    10488
1     1594
Name: 0, dtype: int64


In [41]:
df = normalize(df)

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
X = df.drop(columns = ["0"])
y = df['0']

X_resampled, y_resampled = SMOTE().fit_resample(X,y)

print (X_resampled.shape,y_resampled.shape)

X_train,X_test,y_train,y_test = train_test_split(X_resampled,y_resampled, train_size = 0.7, stratify = y_resampled)

print (X_train.shape,X_test.shape,y_train.shape,y_test.shape)

ValueError: Unknown label type: 'continuous'

In [45]:
pip install imblearn

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.10.1-py3-none-any.whl (226 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
